# <center id="c1"><h1>  🕵️‍♂️ Агенты в LangChain 🦜🔗</h1></center>

### Оглавление ноутбука
<img src='https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEjmGItXgOiUgudJ1R3P6gUfz4pZagEYjiyuQgdcuXVWTLoFFBfw9yBcYoatl5XqLCEbFRwMpYFVd2Zqxi-mAIRg5ZPTVKSl_18QWpKKL1XC3f2_OYbfm8wzQTn2mLE5kePr8ESKbTtT34sg1eboaJS0x49x6AYUFst-3XK8ZHjaNaI-uh2oUg4qI-Bk5Q/s1600/transformers_agent%20%281%29.png' align="right" width="600" height="650" />
<br>

<p><font size="3" face="Arial" font-size="large"><ul type="square">
    
<li><a href="#c1">🚀 Введение </a></li>
<li><a href="#c2">🛠🪚 Инструменты для агентов 🤖</a></li>
<li><a href="#c3">🐍 Python Agent 😎 </a>
<li><a href="#c4">✍️🤖 Создаём первый мини-сервис </a>
<li><a href="#c5"> 🧸 Выводы и заключения ✅</a>
    
</ul></font></p>

### В этом ноутбуке подробнее подробнее познакомимся с использованием `Agents` в `LangChain`.

<div class="alert alert-info">
    
Агенты `LangChain` — это специализированные компоненты в рамках LangChain, которые взаимодействуют с реальным миром. Эти агенты специально разработаны для выполнения четко определенных задач, таких как ответы на вопросы, генерация текста, перевод на другие языки, обобщение текста и пр. <br>
Например, при всех выдающихся результатах LLM в решении различных задач, модель не сможет ответить на такой простой вопрос: "Какой сегодня день недели?" или о каком-то событии, которое произошло месяц назад. Здесь на помощь LLM приходят различные инструменты (tools) взаимодействия с внешним миром, пользоваться которыми ей помогает агент. <br>
Другими словами, **агент это посредник между LLM и Tools**.

Архитектура агента `LangChain` выглядит следующим образом:
* **Прием ввода**: агент получает вводимые пользователем данные на естественном языке.
* **Обработка с помощью LLM**: агент использует LLM для обработки входных данных и формулирования плана действий.
* **Выполнение плана**: агент выполняет разработанный план действий, который может включать взаимодействие с другими инструментами или службами.
* **Доставка выходных данных**: впоследствии агент доставляет выходные данные выполненного плана обратно пользователю.

Существует 2 типа агентов:
* `Toolkits` агенты (агент - ящик инструментов) - заточенные под выполнение конкретных задач или для работы с конкретным API.

* `Generic` агенты (общего назначения) - универсальные, которые способны выполнять множество метазадач.

In [1]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#!pip install langchain langchain_experimental openai tiktoken numexpr -q

In [ ]:
# Для работы в колабе загрузите наш скрипт для использования ChatGPT на сервере курса!
#!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py

In [65]:
# # Если используете ключ от OpenAI, запустите эту ячейку
# from langchain.chat_models import ChatOpenAI

# # os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
# os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# # инициализируем языковую модель
# llm = ChatOpenAI(temperature=0.0)

In [2]:
from utils import ChatOpenAI
# Если используете ключ из курса, запустите эту ячейку
#from utils import ChatOpenAI

#course_api_key= "Введите ваш ключ, полученный в боте курса"
course_api_key = getpass(prompt="Введите ваш ключ, полученный в боте курса")

# инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

Введите ваш ключ, полученный в боте курса ········


 #  <center id="c2"> 🛠🪚  Инструменты для агентов 🤖
    
<div class="alert alert-info">

Для использования агентов нам понадобятся 3 компонента:
* LLM
* Инструменты (Tools)
* Сами агенты

<div class="alert alert-success">

Теперь давайте создадим наш первый `Tool` - калькулятор, используя уже знакомую нам `LLMMathChain`:

In [3]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain.from_llm(llm=llm)

# создаём math tool, задаём имя, описание и функцию
math_tool = Tool(
    name='Калькулятор',
    func=llm_math.run,
    description='Может производить математические расчёты.'
)

# Для передачи в агента, нам нужно поместить инструменты в список
tools = [math_tool]

In [23]:
tools[0].name, tools[0].description

('Калькулятор', 'Может производить математические рассчёты.')

<div class="alert alert-success">

Создадим нашего **первого агента**, передав в него инструменты (пока 1)

In [4]:
from langchain.agents import initialize_agent

first_agent = initialize_agent(
    tools=tools,
    llm=llm,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=3
)

In [25]:
first_agent("Сколько будет (4.5*2.1)+2.2?")



> Entering new AgentExecutor chain...
I need to calculate the result of the given expression.
Action: Калькулятор
Action Input: (4.5*2.1)+2.2
Observation: Answer: 11.650000000000002
Thought:The final answer is 11.650000000000002.
Final Answer: 11.650000000000002

> Finished chain.


{'input': 'Сколько будет (4.5*2.1)+2.2?', 'output': '11.650000000000002'}

In [50]:
# Проверим
(4.5*2.1)+2.2

11.650000000000002

<div class="alert alert-success">
    
Попробуем по сложнее:

In [27]:
first_agent('''У Маши было 3 яблока, Гриша принёс ещё 2 с половиной ящика яблок.
            Сколько всего яблок у ребят, если ящик вмещает 8 яблок?''')



> Entering new AgentExecutor chain...
I need to calculate the total number of apples.
Action: Калькулятор
Action Input: 3 + (2.5 * 8)
Observation: Answer: 23.0
Thought:The final answer is 23.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I made a mistake in the format. I need to provide the correct format.
Final Answer: The total number of apples is 23.

> Finished chain.


{'input': 'У Маши было 3 яблока, Гриша принёс ещё 2 с половиной ящика яблок.\n            Сколько всего яблок у ребят, если ящик вмещает 8 яблок?',
 'output': 'The total number of apples is 23.'}

### А что будет, если мы спросим не математический вопрос?
```python
first_agent("Столица Хорватии?") 
```

Получим ошибку: `ValueError: unknown format from LLM: I'm sorry, but I can only help with math-related questions.`

<div class="alert alert-info">

Это происходит потому, что агент принуждает к использованию инструмента (это не всегда так). Обходной путь, который мы можем реализовать, — это просто добавить еще один инструмент, который может ответить на этот вопрос.

Если мы хотим, чтобы агент мог отвечать на `general knowledge questions` (общие вопросы), можно добавить саму LLM как инструмент `LLM-tool`.


In [36]:
# создаём LLM tool
llm_tool = Tool(
    name='Language Model',
    func=llm.predict,
    description='Отвечает на общие вопросы'
)

# Добавляем его к другим инструментам
tools.append(llm_tool)

In [37]:
first_agent = initialize_agent(
    tools=tools,
    llm=llm,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=3
)

In [38]:
first_agent("Столица Хорватии?")



> Entering new AgentExecutor chain...
I know that the capital of Croatia is Zagreb, but I can also confirm it using the language model.
Action: Language Model
Action Input: "What is the capital of Croatia?"
Observation: The capital of Croatia is Zagreb.
Thought:I now know the final answer
Final Answer: The capital of Croatia is Zagreb.

> Finished chain.


{'input': 'Столица Хорватии?', 'output': 'The capital of Croatia is Zagreb.'}

<div class="alert alert-success">

Видим, что после ввода запроса, агент передаёт его в LLM, и модель решает какой из инструментов лучше использовать.

<div class="alert alert-info">

Иногда из-за особенностей токенизации, модель может ошибаться в ответах на достаточно простые вопросы:

In [19]:
llm.invoke("Сколько букв в слове зачёт?")

AIMessage(content='В слове "зачёт" 6 букв.')

<div class="alert alert-success">

Рассмотрим ещё один способ создания инструмента из Python-функции, при помощи декоратора `tool`.

В функцию обязательно необходимо добавлять описание (`docstring`), чтобы агент понимал для чего её нужно применять!

In [21]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Возвращает длину слова""" # добавляем docstring
    return len(word)


tools = [get_word_length]

<div class="alert alert-success">

Создадим агента с нашим инструментом, и посмотрим как он справится с задачей:

In [23]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    verbose=True,
)

agent("Сколько букв в слове зачёт?")



> Entering new AgentExecutor chain...
I need to find the length of the word "зачёт".
Action: get_word_length
Action Input: "зачёт"
Observation: 5
Thought:The word "зачёт" has 5 letters.
Final Answer: 5

> Finished chain.


{'input': 'Сколько букв в слове зачёт?', 'output': '5'}

# <center id="c3"> 🐍 Python Agent 😎

<div class="alert alert-info">

Если вы видели такие инструменты кодирования, как `GitHub Copilot`, и задавались вопросом, как они работают, то одна из задач, которые они делают, — это заставляют языковую модель написать код, а затем выполнить этот код. Мы можем сделать то же самое, создав агент `Python`. Для этого мы используем LLM, а также инструмент(tool) `PythonREPL`, который представляет собой способ выполнения`Python` кода, немного похожий на `Jupyter Notebook`, поэтому агент может выполнить этот код, используя его, получить некоторые результаты, и эти результаты будут переданы обратно агенту, чтобы он мог решить, что делать дальше.

In [42]:
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.agents import initialize_agent

py_agent = initialize_agent(
    tools=[PythonREPLTool()],
    llm=llm,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=3
)

In [44]:
names = ["Борис", "Олег", "Ия", "Александр", "Зоя"]

In [45]:
py_agent.run(f'Отсортируй имена в списке по длине в порядке убывания и покажи результат: {names}')



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


I can use the `sorted()` function with a custom key to sort the names by length in descending order.
Action: Python_REPL
Action Input: `names = ['Борис', 'Олег', 'Ия', 'Александр', 'Зоя']`
Observation: 
Thought:I can use the `sorted()` function with a custom key to sort the names by length in descending order.
Action: Python_REPL
Action Input: `sorted(names, key=lambda x: len(x), reverse=True)`
Observation: 
Thought:I now know the final answer
Final Answer: ['Александр', 'Борис', 'Олег', 'Зоя', 'Ия']

> Finished chain.


"['Александр', 'Борис', 'Олег', 'Зоя', 'Ия']"

# <center id="c4"> ✍️🤖 Создаём первый мини-сервис

<div class="alert alert-info">
    
Теперь давайте попробуем реализовать "полезного" агента с использованием нескольких инструментов.
В этом примере мы будем использовать инструменты `Wikipedia`, `DuckDuckGo` и `Arxiv`, чтобы создать простой агент для написания эссе. <br> [Здесь](https://python.langchain.com/docs/integrations/tools/) доступен длинный список инструментов, которые агенты могут использовать для взаимодействия с внешним миром.

In [ ]:
#!pip install duckduckgo-search arxiv wikipedia

In [5]:
from langchain.agents import load_tools
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

# инструменты, поддерживаемые LangChain "из коробки", можно подгружать функцией load_tools
tools = load_tools(["arxiv", "wikipedia"], llm=llm)

tools.extend([  
    Tool(  
        name="Search",  
        func=search.run,  
        description="useful for when you need to answer questions about current events."  
    ),  
])

essay_agent = initialize_agent(
    tools, 
    llm, 
    verbose=True, 
    handle_parsing_errors=True,
    max_iterations=5)

In [7]:
prompt = "Write a 1000-word essay on topic: {topic}, use the tools to retrieve the necessary information."  
topic = "Style Quentin Tarantino in cinema"  

print(essay_agent.run(prompt.format(topic=topic)))



> Entering new AgentExecutor chain...
I need to gather information about Quentin Tarantino's style in cinema. I can use Wikipedia to get an overview of his career and arxiv to find any scholarly articles or research papers on his style.
Action: Wikipedia
Action Input: Quentin Tarantino
Observation: Page: Quentin Tarantino
Summary: Quentin Jerome Tarantino (; born March 27, 1963) is an American film director, screenwriter, and actor. His films are characterized by stylized violence, extended dialogue including a pervasive use of profanity, and references to popular culture.
Tarantino began his career as an independent filmmaker with the release of the crime film Reservoir Dogs in 1992. His second film, Pulp Fiction (1994), a dark comedy crime thriller, was a major success with critics and audiences winning numerous awards, including the Palme d'Or and the Academy Award for Best Original Screenplay. In 1996, he appeared in From Dusk till Dawn, also writing the screenplay. Tarantino's t

<div class="alert alert-success">

Агент, в целом, справился с задачей, видно, что сначала он сходил в Википедию, затем в Arxiv, потом выдал требуемое эссэ. 
Но в `Final answer` вместо эссэ на 1000 слов агент записал краткий ответ на вопрос, видимо, нужно получше составить промпт (можете потюнить промпт в качестве самостоятельной работы).
    
Тут мог возникнуть справедливый вопрос: А зачем использовать агента, если можно сделать тоже самое, например, с помощью `RouterChain`?
В отличии от цепочек, где порядок действий жёстко прописан, агент получает на вход запрос (промпт), список доступных инструментов и сам решает какие именно инструменты и в каком порядке применить для решения задачи.

### Так же с помощью `load_tools` можно подгрузить такой интересный tool:
```python
tools = load_tools(["human"])
```

<div class="alert alert-info">

В документации к этому tool сказано:
> Human are AGI so they can certainly be used as a tool to help out AI agent when it is confused.

[Здесь](https://python.langchain.com/docs/integrations/tools/human_tools) можно прочитать про этот инструмент подробнее, пока ещё ИИ не может справиться без нас 🫣

# <center id="c5"> 🧸 Выводы и заключения ✅ 

<div class="alert alert-info">
    
Подводя итог, можем выделить несколько явных преимуществ которые дают агенты `LangChain`:
* **Удобство для пользователя**: агенты `LangChain` специально разработаны так, чтобы быть понятными и доступными даже для разработчиков, у которых нет большого опыта работы со сложными языковыми моделями.
* **Универсальность**: адаптируемость агентов `LangChain` позволяет им находить применение для самых разных сценариев и требований.
* **Расширенные возможности.** Используя возможности языковых моделей, агенты LangChain предоставляют такие функции, как понимание и генерация естественного языка, давая доступ к большому количеству функций.
    
В следующих уроках мы рассмотрим более сложные архитектуры агентов, с памятью, внешней базой знаний и пр.